# Lesson 5: Memory & Sessions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harshit-vibes/lyzr-adk-demo/blob/master/notebooks/05_memory_and_sessions.ipynb)


**🟡 Intermediate · ⏱ 25 min**

---

Agents are **stateless by default** — each call to `agent.run()` is completely isolated. Sessions and memory give your agent **conversational continuity**, letting it remember what was said earlier in a conversation.

## What you will learn

- Understand **session IDs** and how they enable multi-turn conversations
- See exactly what happens **without** session IDs (isolated calls)
- Enable and disable **built-in memory** with `add_memory()` / `remove_memory()`
- Control context window size with **`max_messages`**

## Prerequisites

Before running this notebook, make sure you have completed:

- **Lesson 1** — Studio & `agent.run()` basics
- **Lesson 2** — Providers and models
- **Lesson 3** — Agent lifecycle
- **Lesson 4** — Structured outputs

You also need your **`LYZR_API_KEY`** set as an environment variable:

```bash
export LYZR_API_KEY="your-api-key-here"
```

In [ ]:
!pip install lyzr-adk -q

In [ ]:
import os
import uuid
from lyzr import Studio

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## 1. Stateless by Default

When you call `agent.run()` **without** a `session_id`, each call is completely independent. The agent has no awareness of previous messages — it starts fresh every time.

Think of it like calling a support hotline where the agent picks up with zero context from your previous call. Every interaction is brand new.

This is the default behavior and is intentional — it keeps things predictable and side-effect-free for single-turn use cases. But for conversational agents, you need something more.

In [ ]:
# Create a basic agent
agent = studio.create_agent(
    name="Conversation Agent",
    provider="openai/gpt-4o",
    role="Friendly conversationalist",
    goal="Have natural, coherent conversations",
    instructions="Remember context from our conversation when answering."
)

# Without session_id: isolated calls — agent doesn't remember
r1 = agent.run("My favourite colour is blue.")
print(f"Turn 1: {r1.response}")

r2 = agent.run("What is my favourite colour?")
print(f"Turn 2 (no session): {r2.response}")  # Won't know "blue"

## 2. Session IDs

A **session ID** is a string that groups multiple `agent.run()` calls into a single conversation. When you pass the same `session_id` to multiple calls, the agent can see the full message history for that session.

**Best practice:** Generate a unique session ID per conversation using `uuid.uuid4()`. This guarantees no two conversations accidentally share state.

```python
session_id = str(uuid.uuid4())  # e.g. "3f2a1b4c-..."
```

Under the hood, lyzr-adk associates each message with the session ID and replays the relevant history on each call.

In [ ]:
# Generate a unique session ID for this conversation
session = str(uuid.uuid4())
print(f"Session ID: {session}")

# Now all runs with this session_id share context
r1 = agent.run("My favourite colour is blue.", session_id=session)
print(f"Turn 1: {r1.response}")

r2 = agent.run("What is my favourite colour?", session_id=session)
print(f"Turn 2 (with session): {r2.response}")  # Should say "blue"

r3 = agent.run("Also, I love hiking on weekends.", session_id=session)
r4 = agent.run("Summarize what you know about me.", session_id=session)
print(f"Turn 4 (summary): {r4.response}")

## 3. Starting a New Conversation

To start a **fresh conversation**, simply generate a new UUID. Each session ID is its own isolated thread of conversation — starting a new session does not affect existing ones.

This is useful when:
- A new user opens a chat window
- A user explicitly clicks "New conversation"
- You want to run parallel, independent conversations

Old sessions remain accessible as long as they exist in the backend — you can resume them at any time by reusing the original session ID.

In [ ]:
# New session = fresh slate
new_session = str(uuid.uuid4())
r5 = agent.run("What is my favourite colour?", session_id=new_session)
print(f"New session (knows nothing): {r5.response}")

# Old session still intact
r6 = agent.run("What is my favourite colour?", session_id=session)
print(f"Old session (remembers): {r6.response}")

## 4. Built-in Memory with `add_memory()`

Sessions track message history at the **request level** — each call includes prior messages. But for longer conversations, you may want smarter, persistent context storage. That is what `add_memory()` provides.

```python
agent.add_memory(max_messages=10)
```

The `max_messages` parameter controls the **sliding window** of messages retained in memory:

| `max_messages` | Behavior |
|---|---|
| `5` | Keep the last 5 message exchanges |
| `10` | Keep the last 10 message exchanges (good default) |
| `20+` | Longer context, higher token usage |

Higher values give the agent more context but also consume more tokens per call. Choose based on your use case and cost constraints.

In [ ]:
# Enable memory — agent now retains last 10 messages
agent.add_memory(max_messages=10)
print("Memory enabled!")

memory_session = str(uuid.uuid4())

agent.run("My name is Alice and I'm a software engineer.", session_id=memory_session)
agent.run("I'm working on a Python project for data analysis.", session_id=memory_session)
agent.run("I prefer concise explanations.", session_id=memory_session)

# Now ask something that requires remembering all three facts
summary_response = agent.run(
    "Based on what I've told you, what kind of help would be most useful for me?",
    session_id=memory_session
)
print(f"Response: {summary_response.response}")

## 5. Removing Memory

You can disable memory at any time by calling `agent.remove_memory()`. This is useful when:

- You want to switch the agent back to stateless mode for a specific use case
- You are done with a multi-turn flow and want to reduce token overhead
- You are testing the difference between memory-on and memory-off behavior

After calling `remove_memory()`, subsequent `agent.run()` calls revert to default stateless behavior (unless a session ID is passed).

In [ ]:
agent.remove_memory()
print("Memory disabled.")

# Without memory, context won't persist the same way
no_mem_session = str(uuid.uuid4())
agent.run("My name is Bob.", session_id=no_mem_session)
r = agent.run("What is my name?", session_id=no_mem_session)
print(f"Response (no memory): {r.response}")

## Common Mistake: Different Session IDs for the Same Conversation

The most frequent error when working with sessions is accidentally creating a **new session ID** for each message instead of reusing the same one.

This happens when the session ID is generated inside a loop, or when it is not stored and passed consistently. The result: the agent always starts from scratch, even though you expected it to remember.

The fix is simple — generate the session ID **once**, store it in a variable, and pass that same variable to every `agent.run()` call in the conversation.

In [ ]:
# Wrong: forgetting to reuse the session_id
agent.run("My name is Charlie.")               # session A (auto-generated)
r = agent.run("What is my name?")              # session B (different auto-generated)
print(f"Different sessions: {r.response}")  # Won't know "Charlie"

# Correct: create one session ID and reuse it
conversation = str(uuid.uuid4())
agent.run("My name is Charlie.", session_id=conversation)
r2 = agent.run("What is my name?", session_id=conversation)
print(f"Same session: {r2.response}")          # Knows "Charlie"

## Exercise: Build a Personalized Travel Advisor

Your task is to build a simple travel advisor bot that **remembers user preferences** across multiple turns and uses them to make a personalized recommendation.

**Steps:**

1. Create a travel advisor agent with an appropriate role, goal, and instructions
2. Enable memory with a sensible `max_messages` value
3. Start a conversation session
4. Tell the bot **3 different preferences** across 3 separate `agent.run()` calls (e.g. preferred climate, travel style, budget)
5. Ask the bot for a destination recommendation — it should use all 3 preferences in its answer

Fill in the `...` placeholders below.

In [ ]:
# TODO: Create an agent to act as a personalized travel advisor
travel_bot = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Add memory
travel_bot.add_memory(max_messages=...)

# TODO: Start a conversation session
trip_session = str(uuid.uuid4())

# TODO: Tell the bot 3 preferences over 3 turns
travel_bot.run(..., session_id=trip_session)
travel_bot.run(..., session_id=trip_session)
travel_bot.run(..., session_id=trip_session)

# TODO: Ask for a personalised recommendation
final = travel_bot.run("Based on my preferences, recommend a destination for my next trip.", session_id=trip_session)
print(final.response)

## Summary

### Session ID vs. Memory

| Feature | `session_id` | `add_memory(max_messages=N)` |
|---|---|---|
| **What it does** | Groups calls into one conversation thread | Stores and replays recent message history |
| **Scope** | Per-call parameter | Agent-level setting |
| **Context retained** | All messages in the session | Last N messages (sliding window) |
| **When to use** | Any multi-turn conversation | When you need persistent, bounded context |
| **How to disable** | Omit the parameter | Call `agent.remove_memory()` |

### Key Takeaways

- Without a `session_id`, every `agent.run()` call is completely isolated — the agent has no memory of previous messages.
- Generate session IDs with `str(uuid.uuid4())` to ensure uniqueness per conversation.
- A new UUID starts a fresh conversation; the old session remains accessible.
- `add_memory(max_messages=N)` enables a sliding window of context that persists across calls.
- Always store your session ID in a variable and reuse it — do not regenerate it between turns.

## Next Steps

You now know how to give your agents conversational memory. In the next lesson, you will learn how to extend agent capabilities beyond conversation.

**Lesson 6: Custom Tools & Functions** — teach your agent to call external APIs, run Python functions, and interact with the real world.

Head to `06_custom_tools_and_functions.ipynb` to continue.